<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Set-up-the-Environment-and-Upload-data" data-toc-modified-id="Set-up-the-Environment-and-Upload-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Set up the Environment and Upload data</a></span></li><li><span><a href="#Initial-Exploration-and-Pre-processing" data-toc-modified-id="Initial-Exploration-and-Pre-processing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Initial Exploration and Pre-processing</a></span><ul class="toc-item"><li><span><a href="#DisGeNET-:-Gene-Disease-Association-(GDA)" data-toc-modified-id="DisGeNET-:-Gene-Disease-Association-(GDA)-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>DisGeNET : Gene Disease Association (GDA)</a></span></li><li><span><a href="#Data-pre-processing" data-toc-modified-id="Data-pre-processing-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Data pre-processing</a></span></li><li><span><a href="#Exploratory-Analysis" data-toc-modified-id="Exploratory-Analysis-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Exploratory Analysis</a></span></li></ul></li><li><span><a href="#Bibliografia" data-toc-modified-id="Bibliografia-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Bibliografia</a></span></li></ul></div>

In [53]:
#packages
import numpy as np
import pandas as pd
from tdc.multi_pred import GDA
from IPython.display import HTML

## Set up the Environment and Upload data

<div style="text-align: justify;">
To ensure that all the project's dependencies are installed correctly, we recommend using a Conda environment. You can easily create the environment with the project's dependencies from the env.yml file included in this repository.

In [54]:
# To use the DisGeNET dataset with TDC platform, we use the following code:
data = GDA(name='DisGeNET')

print("Data uploading completed :)")

Found local copy...
Loading...
Done!


Data uploading completed :)


## Initial Exploration and Pre-processing

<div style="text-align: justify;">
This section contains a comprehensive overview of the assigned data, starting with a detailed description and characterization based on existing documentation and literature. It includes a concise summary of the data's characteristics derived from an initial exploratory analysis, highlighting key statistics and observations. The section also describes the steps taken for data preparation and pre-processing, providing justification for each choice to ensure the data is clean, transformed, and organized for further analysis. Additionally, initial exploratory graphs are presented to visually illustrate the main characteristics of the data, helping to understand patterns, trends, and potential issues within the dataset.
 </div>

### DisGeNET : Gene Disease Association (GDA) 

In [55]:
HTML('<div style="display: flex; justify-content: center;"><img src="https://www.ensembl.info/wp-content/uploads/2021/04/disgenet-logo-horizontal.png" width="400"/></div>')

The study of **gene-disease associations is a crucial step in understanding the etiology of diseases**. Identifying the link between genes and diseases makes it possible to better understand the cause of the disease and therefore to develop strategies to combat it. This process is fundamental to the advancement of medicine and the development of new therapies. [1]

    

**Therapeutics Data Commons (TDC) is a database created to promote the discovery of therapies for diseases**, especially those that are difficult to treat. It was developed by the National Institutes of Health with the aim of gathering genomic and clinical data from various sources to support biomedical research and the development of new treatments. [2]


**GDA quantify the relation among a pair of gene-disease** and is one of the core concepts of DisGeNet platform that are integrated on TDC. DisGeNET is a discovery platform containing one of the largest publicly available collections of genes and variants associated to human diseases. **DisGeNET integrates data from expert curated repositories, Genome-Wide Association Study catalogues, animal models and the scientific literature**. DisGeNET data are homogeneously annotated with controlled vocabularies and community-driven ontologies. TDC uses the curated subset from UNIPROT, CGI, ClinGen, Genomics England, CTD (human subset), PsyGeNET, and Orphanet. TDC maps disease ID to disease definition through MedGen and maps GeneID to uniprot amino acid sequence. [3]


This dataset contains **52,476 gene-disease pairs:** 7,399 genes and 7,095 diseases.                                   
As we can see by de code bellow we have an object with 52,476 rows and 5 columns: *'Gene_ID'*, *'Gene'*, *'Disease_ID'*, *'Disease'*, *'Y'*. 

- **Gene_ID** column contains all unique identifiers for genes.
- **Gene** column contains the sequences of amino acides corresponding to each gene_ID through Uniprot platform.
- **Disease_ID** column contains all unique identifiers for diseases.
- **Disease** column contains the disease definition corresponding to each disease_ID through MedGen-NCBI platform.
- **Y** column is the target feature and represents the probability of association for a pair gene-disease. This column have values between 0 and 1.

As our taregt feature is a numeric one, our work will be a regression problem when the main task is: *Given the disease description and the amino acid sequence of the gene, predict their association*.

In [56]:
#Inspecting DisGeNet dataset
full_data = data.get_data()
full_data.info()
full_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52476 entries, 0 to 52475
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Gene_ID     52476 non-null  int64  
 1   Gene        52476 non-null  object 
 2   Disease_ID  52476 non-null  object 
 3   Disease     52476 non-null  object 
 4   Y           52476 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 2.0+ MB


,Gene_ID,Gene,Disease_ID,Disease,Y
0,1,MSMLVVFLLLWGVTWGPVTEAAIFYETQPSLWAESESLLKPLANVT...,C0019209,Hepatomegaly: Abnormal enlargement of the liver.,0.30
1,1,MSMLVVFLLLWGVTWGPVTEAAIFYETQPSLWAESESLLKPLANVT...,C0036341,Schizophrenia: Schizophrenia is highly heritab...,0.30
2,2,MGKNKLLHPSLVLLLLVLLPTDASVSGKPQYMVLVPSLLHTETTEK...,C0002395,Alzheimer's Disease: Alzheimer disease is the ...,0.50
3,2,MGKNKLLHPSLVLLLLVLLPTDASVSGKPQYMVLVPSLLHTETTEK...,C0007102,Malignant tumor of colon: A primary or metasta...,0.31
4,2,MGKNKLLHPSLVLLLLVLLPTDASVSGKPQYMVLVPSLLHTETTEK...,C0009375,Colonic Neoplasms: A benign or malignant neopl...,0.30


### Data pre-processing

In [57]:
# Importing necessary libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split


#### Checking for Missing Values
Although the dataset is curated, we explicitly verify for null values to ensure data integrity:

In [58]:
print("Checking for missing values:")
missing_values = full_data.isnull().sum()
print(missing_values)

if missing_values.sum() == 0:
    print("No missing values found. The dataset is clean and ready for preprocessing :)")


Checking for missing values:
Gene_ID       0
Gene          0
Disease_ID    0
Disease       0
Y             0
dtype: int64
No missing values found. The dataset is clean and ready for preprocessing :)


#### Encoding Disease_ID
Disease_ID is a categorical feature. Encoding it to numerical values ensures compatibility with machine learning models.

In [59]:
# We use LabelEncoder to convert each unique Disease_ID into a corresponding numeric code.
le_disease = LabelEncoder()
full_data['Disease_ID_encoded'] = le_disease.fit_transform(full_data['Disease_ID'])

#### Mapping Encoded Labels Back to Disease Names
For interpretability and visualization purposes, we create a reverse mapping to convert numeric codes back to disease names.

In [60]:
disease_reverse_mapping = dict(zip(full_data['Disease_ID_encoded'], full_data['Disease']))
print("Example of Disease_ID_encoded to Disease mapping:")
for key, value in list(disease_reverse_mapping.items())[:5]:  # Displaying the first 5 mappings
    print(f"{key}: {value}")

Example of Disease_ID_encoded to Disease mapping:
635: Hepatomegaly: Abnormal enlargement of the liver.
1308: Schizophrenia: Schizophrenia is highly heritable, as shown by family, twin, and adoption studies. For example, for identical twins, if one twin develops schizophrenia, the other twin has about a 50% chance of also developing the disease. The risk of the general population developing the schizophrenia is about 0.3-0.7% worldwide. The search for “schizophrenia genes” has been elusive. Initial linkage studies looked at parts of the genome associated with schizophrenia, and many candidate genes were identified, including APOE, COMT, DAO, DRD1, DRD2, DRD4, DTNBP1, GABRB2, GRIN2B, HP, IL1B, MTHFR, PLXNA2, SLC6A4, TP53, and TPH1. However, some of these have later been questioned. Microdeletions and microduplications have been found to be three times more common in individuals with schizophrenia, compared to controls. Because these deletions and duplications are in genes that are overe

#### Normalizing the Target (Y)
Normalizing the target values ensures they are on the same scale, which can improve the performance of machine learning algorithms.

In [61]:
scaler = MinMaxScaler()
full_data['Y_normalized'] = scaler.fit_transform(full_data[['Y']])

#### Splitting the Dataset into Train, Validation, and Test Sets
Dividing the dataset allows us to train, validate, and test the model on different subsets, ensuring a robust evaluation.

In [62]:
# We follow an 80-10-10 split for train, validation, and test sets.
train_data, test_data = train_test_split(full_data, test_size=0.2, random_state=42)
train_data, valid_data = train_test_split(train_data, test_size=0.1, random_state=42)


#### Preparing Features and Target
Features are the inputs for our model, while the target is the output. Here, we select the appropriate columns for both. Defining features and target variables is crucial for supervised learning. 


In [63]:
# The features include Gene_ID and Disease_ID_encoded, while the target is Y_normalized.
X_train = train_data[['Gene_ID', 'Disease_ID_encoded']]
y_train = train_data['Y_normalized']

X_valid = valid_data[['Gene_ID', 'Disease_ID_encoded']]
y_valid = valid_data['Y_normalized']

X_test = test_data[['Gene_ID', 'Disease_ID_encoded']]
y_test = test_data['Y_normalized']


#### Validating the Pre-processing
Ensuring that the splits have the expected sizes and that the features and target are properly prepared.

In [64]:
print(f"Training set: {X_train.shape}, Validation set: {X_valid.shape}, Test set: {X_test.shape}")
print("First few rows of X_train:")
print(X_train.head())

Training set: (37782, 2), Validation set: (4198, 2), Test set: (10496, 2)
First few rows of X_train:
       Gene_ID  Disease_ID_encoded
24634     5055                6910
40534    10293                 415
8064      1509                4606
29814     6323                4624
25108     5160                1254


#### Preparing for Visualization
For exploratory data analysis, we add a column with the disease names to replace numeric codes when plotting.

In [65]:
X_train['Disease_Label'] = X_train['Disease_ID_encoded'].map(disease_reverse_mapping)
X_valid['Disease_Label'] = X_valid['Disease_ID_encoded'].map(disease_reverse_mapping)
X_test['Disease_Label'] = X_test['Disease_ID_encoded'].map(disease_reverse_mapping)

print("First few rows of X_train with disease labels:")
print(X_train[['Gene_ID', 'Disease_Label']].head())

First few rows of X_train with disease labels:
       Gene_ID                                      Disease_Label
24634     5055  Mammary Carcinoma, Human: Cancer of the human ...
40534    10293  Dwarfism: A genetic or pathological condition ...
8064      1509  Neuronal Ceroid Lipofuscinosis, Congenital: CL...
29814     6323  Migraine, Familial Hemiplegic, 3: Familial hem...
25108     5160  Pyruvate Metabolism, Inborn Errors: Hereditary...


### Exploratory Analysis

## Bibliografia

[1] Opap, K., & Mulder, N. (2017). Recent advances in predicting gene–disease associations. F1000Research, 6, 578. https://doi.org/10.12688/f1000research.10788.1
    
[2] Therapeutics Data Commons. (2024). TDC. https://tdcommons.ai/

[3] Piñero, J., Ramírez-Anguita, J. M., Saüch-Pitarch, J., Ronzano, F., Centeno, E., Sanz, F., & Furlong, L. I. (2019). The DisGeNET knowledge platform for disease genomics: 2019 update. Nucleic Acids Research, 48(D1). https://doi.org/10.1093/nar/gkz1021